In [1]:
import numpy as np
import pandas as pd
import math
pd.set_option('display.max_columns', 100)
pd.options.display.float_format = '{:.2f}'.format

# Outline

###  Part A. Affordable  Rentals
 1. Bring in 2016 PUMS data
 2. Assign county_id to each puma
 3. Bring in and clean, reformat county AMIs data
 4. Add variables for cost limits for ELI, VLI, LI, MI, and HI by bedroom count for each county
 5. Add count dummies for each unit based on income and unit type
 

###  Part B. Affordable Owning
 1. Calculate total monthly housing cost by adding monthly first mortgage, second/junior mortgage, and condo fees
 2. Add count dummies for each unit based on income and unit type

### Part C. Weighting Sample to get estimates
 1. Multiply each dummy by the the household weight variable
 2. Calculte standard errors for each estimate
 
### Part D. Aggregating at PUMA Level
 1. Add dummy variables for vacant "for-rent," vacant "for-sale," and recently moved in and create second dataset
 2. Aggregate at PUMA level for each dataset and export

## Part A1. Bring in 2016 PUMS data

In [2]:
variable_types = {"BDSP":"float","RMSP":"str","MV":"float","WGTP":"float","TAXP":"float"}
columns = ["PUMA","RT","BDSP","BLD","RNTP","MRGP","SMP","TAXP","CONP","TEN","VACS","VALP","GRPIP","GRNTP","HINCP","MV","WGTP"]
pums_df=pd.read_csv("2016_pums.csv", delimiter=",",usecols=columns,dtype=variable_types)
pums_df

,RT,PUMA,WGTP,BDSP,BLD,CONP,MRGP,RNTP,SMP,TEN,VACS,VALP,GRNTP,GRPIP,HINCP,MV,TAXP
0,H,10702,102.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,160000.00,nan,nan,54000.00,6.00,25.00
1,H,6514,122.00,4.00,2.00,0.00,1000.00,nan,nan,1.00,nan,290000.00,nan,nan,41500.00,7.00,28.00
2,H,3759,58.00,3.00,2.00,0.00,2000.00,nan,nan,1.00,nan,700000.00,nan,nan,204000.00,3.00,65.00
3,H,5917,73.00,2.00,2.00,300.00,2100.00,nan,nan,1.00,nan,400000.00,nan,nan,130000.00,1.00,30.00
4,H,3739,52.00,1.00,6.00,nan,nan,1100.00,nan,3.00,nan,nan,1150.00,18.00,76000.00,3.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156077,H,3753,69.00,2.00,2.00,0.00,1300.00,nan,nan,1.00,nan,350000.00,nan,nan,34800.00,6.00,38.00
156078,H,3734,45.00,1.00,8.00,nan,nan,700.00,nan,3.00,nan,nan,780.00,21.00,44300.00,7.00,nan
156079,H,702,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
156080,H,3730,137.00,2.00,9.00,nan,nan,2600.00,nan,3.00,nan,nan,2600.00,42.00,74800.00,4.00,nan


In [3]:
pums_df.rename(columns={"RT":"record_type","BDSP":"number_bedrooms",
    "BLD":"units_in_structure",
    "RNTP":"monthly_rent",
    "MRGP":"first_mortgage",
    "SMP":"second_mortgage",
    "CONP":"condo_fee",
    "TEN":"tenure",
    "VACS":"vacancy_status",
    "VALP":"property_value",
    "GRPIP":"gross_rent_pct_of_income",
    "GRNTP":"gross_rent",
    "HINCP":"hh_income",
    "TAXP":"prop_tax",
    "WGTP":"weight",
    "MV":"moved_in"}, inplace=True)
pums_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax
0,H,10702,102.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,160000.00,nan,nan,54000.00,6.00,25.00
1,H,6514,122.00,4.00,2.00,0.00,1000.00,nan,nan,1.00,nan,290000.00,nan,nan,41500.00,7.00,28.00
2,H,3759,58.00,3.00,2.00,0.00,2000.00,nan,nan,1.00,nan,700000.00,nan,nan,204000.00,3.00,65.00
3,H,5917,73.00,2.00,2.00,300.00,2100.00,nan,nan,1.00,nan,400000.00,nan,nan,130000.00,1.00,30.00
4,H,3739,52.00,1.00,6.00,nan,nan,1100.00,nan,3.00,nan,nan,1150.00,18.00,76000.00,3.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156077,H,3753,69.00,2.00,2.00,0.00,1300.00,nan,nan,1.00,nan,350000.00,nan,nan,34800.00,6.00,38.00
156078,H,3734,45.00,1.00,8.00,nan,nan,700.00,nan,3.00,nan,nan,780.00,21.00,44300.00,7.00,nan
156079,H,702,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
156080,H,3730,137.00,2.00,9.00,nan,nan,2600.00,nan,3.00,nan,nan,2600.00,42.00,74800.00,4.00,nan


## Part A2. Assign county id to each puma

In [4]:
# load in crosswalk file
crosswalk_df=pd.read_csv("C:/Users/Terner GSR/Box/Cost of Doing Nothing/Climate Impacts/Data/PUMS/PUMA_County_Crosswalk_v2.csv", delimiter=",")
crosswalk_df

,PUMA,county1,county2,county3,county4,county5,county6,county7
0,101,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
1,102,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
2,103,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
3,104,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
4,105,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
260,11103,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
261,11104,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
262,11105,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
263,11106,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# add county name column to puma file
puma_county_df=pums_df.merge(crosswalk_df, how='left', left_on = "PUMA",right_on = "PUMA")
puma_county_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7
0,H,10702,102.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,160000.00,nan,nan,54000.00,6.00,25.00,Tulare CA,NaN,NaN,NaN,NaN,NaN,NaN
1,H,6514,122.00,4.00,2.00,0.00,1000.00,nan,nan,1.00,nan,290000.00,nan,nan,41500.00,7.00,28.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN
2,H,3759,58.00,3.00,2.00,0.00,2000.00,nan,nan,1.00,nan,700000.00,nan,nan,204000.00,3.00,65.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN
3,H,5917,73.00,2.00,2.00,300.00,2100.00,nan,nan,1.00,nan,400000.00,nan,nan,130000.00,1.00,30.00,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN
4,H,3739,52.00,1.00,6.00,nan,nan,1100.00,nan,3.00,nan,nan,1150.00,18.00,76000.00,3.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156077,H,3753,69.00,2.00,2.00,0.00,1300.00,nan,nan,1.00,nan,350000.00,nan,nan,34800.00,6.00,38.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN
156078,H,3734,45.00,1.00,8.00,nan,nan,700.00,nan,3.00,nan,nan,780.00,21.00,44300.00,7.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN
156079,H,702,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Butte CA,NaN,NaN,NaN,NaN,NaN,NaN
156080,H,3730,137.00,2.00,9.00,nan,nan,2600.00,nan,3.00,nan,nan,2600.00,42.00,74800.00,4.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# remove " CA" from end of each county name
puma_county_df['County'] = puma_county_df['county1'].str.replace(r' CA', '')
puma_county_df['county2'] = puma_county_df['county2'].str.replace(r' CA', '')
puma_county_df['county3'] = puma_county_df['county3'].str.replace(r' CA', '')
puma_county_df['county4'] = puma_county_df['county4'].str.replace(r' CA', '')
puma_county_df['county5'] = puma_county_df['county5'].str.replace(r' CA', '')
puma_county_df['county6'] = puma_county_df['county6'].str.replace(r' CA', '')
puma_county_df['county7'] = puma_county_df['county7'].str.replace(r' CA', '')
puma_county_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County
0,H,10702,102.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,160000.00,nan,nan,54000.00,6.00,25.00,Tulare CA,NaN,NaN,NaN,NaN,NaN,NaN,Tulare
1,H,6514,122.00,4.00,2.00,0.00,1000.00,nan,nan,1.00,nan,290000.00,nan,nan,41500.00,7.00,28.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside
2,H,3759,58.00,3.00,2.00,0.00,2000.00,nan,nan,1.00,nan,700000.00,nan,nan,204000.00,3.00,65.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles
3,H,5917,73.00,2.00,2.00,300.00,2100.00,nan,nan,1.00,nan,400000.00,nan,nan,130000.00,1.00,30.00,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange
4,H,3739,52.00,1.00,6.00,nan,nan,1100.00,nan,3.00,nan,nan,1150.00,18.00,76000.00,3.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156077,H,3753,69.00,2.00,2.00,0.00,1300.00,nan,nan,1.00,nan,350000.00,nan,nan,34800.00,6.00,38.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles
156078,H,3734,45.00,1.00,8.00,nan,nan,700.00,nan,3.00,nan,nan,780.00,21.00,44300.00,7.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles
156079,H,702,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Butte CA,NaN,NaN,NaN,NaN,NaN,NaN,Butte
156080,H,3730,137.00,2.00,9.00,nan,nan,2600.00,nan,3.00,nan,nan,2600.00,42.00,74800.00,4.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles


## Part A3. Bring in and clean, reformat county AMIs data

In [7]:
#bring in 2018 county AMIs file
columns=["County","Income_Category","1","2","4","6","8"]
df = pd.read_csv("2016_amis.csv", delimiter=",", usecols=columns)
df

,County,Income_Category,1,2,4,6,8
0,Alameda County\n4-Per,Extremely Low,20500,23400,29250,33950,40890
1,Alameda County\n4-Per,Very Low Income,34150,39000,48750,56550,64350
2,Alameda County\n4-Per,Low Income,52650,60150,75150,87200,99200
3,Alameda County\n4-Per,Median Income,65500,74900,93600,108600,123550
4,Alameda County\n4-Per,Moderate Income,78600,89850,112300,130250,148250
...,...,...,...,...,...,...,...
285,Yuba County\n4-Person,Extremely Low,12500,16020,24300,32580,38900
286,Yuba County\n4-Person,Very Low Income,20800,23800,29700,34500,39250
287,Yuba County\n4-Person,Low Income,33250,38000,47500,55100,62700
288,Yuba County\n4-Person,Median Income,41600,47500,59400,68900,78400


In [8]:
#clean county names variable
df['County'] = df['County'].str.replace(r' County\n4-Person', '')
df['County'] = df['County'].str.replace(r' County\n4-Perso', '')
df['County'] = df['County'].str.replace(r' County\n4-Pers', '')
df['County'] = df['County'].str.replace(r' County\n4-Per', '')
df['County'] = df['County'].str.replace(r' County\n4-Pe', '')
df['County'] = df['County'].str.replace(r' County\n4-P', '')
df['County'] = df['County'].str.replace(r' County\n4-', '')
df['County'] = df['County'].str.replace(r' County\n4', '')
df['County'] = df['County'].str.replace(r' County\n', '')
df['County'] = df['County'].str.replace(r' County\ ' , '')
df['County'] = df['County'].str.replace(r' County', '')
df['County'] = df['County'].str.replace(r' Count', '')
df['County'] = df['County'].str.replace(r' Coun', '')
df

,County,Income_Category,1,2,4,6,8
0,Alameda,Extremely Low,20500,23400,29250,33950,40890
1,Alameda,Very Low Income,34150,39000,48750,56550,64350
2,Alameda,Low Income,52650,60150,75150,87200,99200
3,Alameda,Median Income,65500,74900,93600,108600,123550
4,Alameda,Moderate Income,78600,89850,112300,130250,148250
...,...,...,...,...,...,...,...
285,Yuba,Extremely Low,12500,16020,24300,32580,38900
286,Yuba,Very Low Income,20800,23800,29700,34500,39250
287,Yuba,Low Income,33250,38000,47500,55100,62700
288,Yuba,Median Income,41600,47500,59400,68900,78400


In [9]:
#create df for each income level and calculate max rents, rename variables
eli_df = df[df.Income_Category =="Extremely Low"].copy()
eli_df["1"]=eli_df["1"]*0.025
eli_df["2"]=eli_df["2"]*0.025
eli_df["4"]=eli_df["4"]*0.025
eli_df["6"]=eli_df["6"]*0.025
eli_df["8"]=eli_df["8"]*0.025
eli_df.rename(columns={"1":"ELI_studio","2":"ELI_1_br","4":"ELI_2_br","6":"ELI_3_br","8":"ELI_4_br"}, inplace=True)
eli_df.drop('Income_Category', axis=1, inplace=True)
eli_df

,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br
0,Alameda,512.50,585.00,731.25,848.75,1022.25
5,Alpine,453.75,518.75,647.50,814.50,1022.25
10,Amador,380.00,435.00,607.50,814.50,1022.25
15,Butte,310.00,400.50,607.50,814.50,972.50
20,Calaveras,368.75,421.25,607.50,814.50,1022.25
25,Colusa,310.00,400.50,607.50,814.50,972.50
30,Contra Costa,512.50,585.00,731.25,848.75,1022.25
35,Del Norte,310.00,400.50,607.50,814.50,972.50
40,El Dorado,400.00,457.50,607.50,814.50,1022.25
45,Fresno,310.00,400.50,607.50,814.50,972.50


In [10]:
vli_df = df[df.Income_Category =="Very Low Income"].copy()
vli_df["1"]=vli_df["1"]*0.025
vli_df["2"]=vli_df["2"]*0.025
vli_df["4"]=vli_df["4"]*0.025
vli_df["6"]=vli_df["6"]*0.025
vli_df["8"]=vli_df["8"]*0.025
vli_df.rename(columns={"1":"VLI_studio","2":"VLI_1_br","4":"VLI_2_br","6":"VLI_3_br","8":"VLI_4_br"}, inplace=True)
vli_df.drop('Income_Category', axis=1, inplace=True)
vli_df

,County,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br
1,Alameda,853.75,975.00,1218.75,1413.75,1608.75
6,Alpine,756.25,865.00,1080.00,1253.75,1426.25
11,Amador,633.75,723.75,903.75,1048.75,1193.75
16,Butte,516.25,590.00,736.25,855.00,972.50
21,Calaveras,615.00,702.50,877.50,1018.75,1158.75
26,Colusa,516.25,590.00,736.25,855.00,972.50
31,Contra Costa,853.75,975.00,1218.75,1413.75,1608.75
36,Del Norte,516.25,590.00,736.25,855.00,972.50
41,El Dorado,666.25,761.25,951.25,1103.75,1256.25
46,Fresno,516.25,590.00,736.25,855.00,972.50


In [11]:
li_df = df[df.Income_Category =="Low Income"].copy()
li_df["1"]=li_df["1"]*0.025
li_df["2"]=li_df["2"]*0.025
li_df["4"]=li_df["4"]*0.025
li_df["6"]=li_df["6"]*0.025
li_df["8"]=li_df["8"]*0.025
li_df.rename(columns={"1":"LI_studio","2":"LI_1_br","4":"LI_2_br","6":"LI_3_br","8":"LI_4_br"}, inplace=True)
li_df.drop('Income_Category', axis=1, inplace=True)
li_df

,County,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br
2,Alameda,1316.25,1503.75,1878.75,2180.00,2480.00
7,Alpine,1152.50,1316.25,1645.00,1908.75,2172.50
12,Amador,1012.50,1157.50,1446.25,1678.75,1910.00
17,Butte,825.00,942.50,1177.50,1366.25,1555.00
22,Calaveras,983.75,1123.75,1403.75,1628.75,1853.75
27,Colusa,825.00,942.50,1177.50,1366.25,1555.00
32,Contra Costa,1316.25,1503.75,1878.75,2180.00,2480.00
37,Del Norte,825.00,942.50,1177.50,1366.25,1555.00
42,El Dorado,1066.25,1218.75,1522.50,1766.25,2010.00
47,Fresno,825.00,942.50,1177.50,1366.25,1555.00


In [12]:
mi_df = df[df.Income_Category =="Median Income"].copy()
mi_df["1"]=mi_df["1"]*0.025
mi_df["2"]=mi_df["2"]*0.025
mi_df["4"]=mi_df["4"]*0.025
mi_df["6"]=mi_df["6"]*0.025
mi_df["8"]=mi_df["8"]*0.025
mi_df.rename(columns={"1":"MI_studio","2":"MI_1_br","4":"MI_2_br","6":"MI_3_br","8":"MI_4_br"}, inplace=True)
mi_df.drop('Income_Category', axis=1, inplace=True)
mi_df

,County,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br
3,Alameda,1637.50,1872.50,2340.00,2715.00,3088.75
8,Alpine,1661.25,1897.50,2372.50,2752.50,3131.25
13,Amador,1265.00,1446.25,1807.50,2096.25,2386.25
18,Butte,1031.25,1177.50,1472.50,1707.50,1943.75
23,Calaveras,1228.75,1403.75,1755.00,2036.25,2316.25
28,Colusa,1031.25,1177.50,1472.50,1707.50,1943.75
33,Contra Costa,1637.50,1872.50,2340.00,2715.00,3088.75
38,Del Norte,1031.25,1177.50,1472.50,1707.50,1943.75
43,El Dorado,1331.25,1522.50,1902.50,2207.50,2511.25
48,Fresno,1031.25,1177.50,1472.50,1707.50,1943.75


In [13]:
moi_df = df[df.Income_Category =="Moderate Income"].copy()
moi_df["1"]=moi_df["1"]*0.025
moi_df["2"]=moi_df["2"]*0.025
moi_df["4"]=moi_df["4"]*0.025
moi_df["6"]=moi_df["6"]*0.025
moi_df["8"]=moi_df["8"]*0.025
moi_df.rename(columns={"1":"MoI_studio","2":"MoI_1_br","4":"MoI_2_br","6":"MoI_3_br","8":"MoI_4_br"}, inplace=True)
moi_df.drop('Income_Category', axis=1, inplace=True)
moi_df

,County,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
4,Alameda,1965.00,2246.25,2807.50,3256.25,3706.25
9,Alpine,1993.75,2277.50,2847.50,3302.50,3758.75
14,Amador,1517.50,1735.00,2168.75,2516.25,2862.50
19,Butte,1237.50,1413.75,1767.50,2050.00,2332.50
24,Calaveras,1473.75,1685.00,2106.25,2443.75,2780.00
29,Colusa,1237.50,1413.75,1767.50,2050.00,2332.50
34,Contra Costa,1965.00,2246.25,2807.50,3256.25,3706.25
39,Del Norte,1237.50,1413.75,1767.50,2050.00,2332.50
44,El Dorado,1597.50,1826.25,2282.50,2647.50,3012.50
49,Fresno,1237.50,1413.75,1767.50,2050.00,2332.50


In [14]:
#combine dfs
all_amis_df=eli_df.merge(vli_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(li_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(mi_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(moi_df, how='left', left_on = "County",right_on = "County")
all_amis_df

,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
0,Alameda,512.50,585.00,731.25,848.75,1022.25,853.75,975.00,1218.75,1413.75,1608.75,1316.25,1503.75,1878.75,2180.00,2480.00,1637.50,1872.50,2340.00,2715.00,3088.75,1965.00,2246.25,2807.50,3256.25,3706.25
1,Alpine,453.75,518.75,647.50,814.50,1022.25,756.25,865.00,1080.00,1253.75,1426.25,1152.50,1316.25,1645.00,1908.75,2172.50,1661.25,1897.50,2372.50,2752.50,3131.25,1993.75,2277.50,2847.50,3302.50,3758.75
2,Amador,380.00,435.00,607.50,814.50,1022.25,633.75,723.75,903.75,1048.75,1193.75,1012.50,1157.50,1446.25,1678.75,1910.00,1265.00,1446.25,1807.50,2096.25,2386.25,1517.50,1735.00,2168.75,2516.25,2862.50
3,Butte,310.00,400.50,607.50,814.50,972.50,516.25,590.00,736.25,855.00,972.50,825.00,942.50,1177.50,1366.25,1555.00,1031.25,1177.50,1472.50,1707.50,1943.75,1237.50,1413.75,1767.50,2050.00,2332.50
4,Calaveras,368.75,421.25,607.50,814.50,1022.25,615.00,702.50,877.50,1018.75,1158.75,983.75,1123.75,1403.75,1628.75,1853.75,1228.75,1403.75,1755.00,2036.25,2316.25,1473.75,1685.00,2106.25,2443.75,2780.00
5,Colusa,310.00,400.50,607.50,814.50,972.50,516.25,590.00,736.25,855.00,972.50,825.00,942.50,1177.50,1366.25,1555.00,1031.25,1177.50,1472.50,1707.50,1943.75,1237.50,1413.75,1767.50,2050.00,2332.50
6,Contra Costa,512.50,585.00,731.25,848.75,1022.25,853.75,975.00,1218.75,1413.75,1608.75,1316.25,1503.75,1878.75,2180.00,2480.00,1637.50,1872.50,2340.00,2715.00,3088.75,1965.00,2246.25,2807.50,3256.25,3706.25
7,Del Norte,310.00,400.50,607.50,814.50,972.50,516.25,590.00,736.25,855.00,972.50,825.00,942.50,1177.50,1366.25,1555.00,1031.25,1177.50,1472.50,1707.50,1943.75,1237.50,1413.75,1767.50,2050.00,2332.50
8,El Dorado,400.00,457.50,607.50,814.50,1022.25,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50
9,Fresno,310.00,400.50,607.50,814.50,972.50,516.25,590.00,736.25,855.00,972.50,825.00,942.50,1177.50,1366.25,1555.00,1031.25,1177.50,1472.50,1707.50,1943.75,1237.50,1413.75,1767.50,2050.00,2332.50


## Part A4. Add count dummies for each unit based on income and unit type

In [15]:
# merge datasets
units_df=puma_county_df.merge(all_amis_df, how='left', left_on = "County",right_on = "County")
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
0,H,10702,102.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,160000.00,nan,nan,54000.00,6.00,25.00,Tulare CA,NaN,NaN,NaN,NaN,NaN,NaN,Tulare,310.00,400.50,607.50,814.50,972.50,516.25,590.00,736.25,855.00,972.50,825.00,942.50,1177.50,1366.25,1555.00,1031.25,1177.50,1472.50,1707.50,1943.75,1237.50,1413.75,1767.50,2050.00,2332.50
1,H,6514,122.00,4.00,2.00,0.00,1000.00,nan,nan,1.00,nan,290000.00,nan,nan,41500.00,7.00,28.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,607.50,814.50,1022.25,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75
2,H,3759,58.00,3.00,2.00,0.00,2000.00,nan,nan,1.00,nan,700000.00,nan,nan,204000.00,3.00,65.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2292.50,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25
3,H,5917,73.00,2.00,2.00,300.00,2100.00,nan,nan,1.00,nan,400000.00,nan,nan,130000.00,1.00,30.00,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,512.50,585.00,731.25,848.75,1022.25,853.75,975.00,1218.75,1413.75,1608.75,1365.00,1560.00,1950.00,2262.50,2575.00,1526.25,1743.75,2180.00,2528.75,2877.50,1831.25,2092.50,2616.25,3035.00,3453.75
4,H,3739,52.00,1.00,6.00,nan,nan,1100.00,nan,3.00,nan,nan,1150.00,18.00,76000.00,3.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2292.50,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156077,H,3753,69.00,2.00,2.00,0.00,1300.00,nan,nan,1.00,nan,350000.00,nan,nan,34800.00,6.00,38.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2292.50,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25
156078,H,3734,45.00,1.00,8.00,nan,nan,700.00,nan,3.00,nan,nan,780.00,21.00,44300.00,7.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2292.50,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25
156079,H,702,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Butte CA,NaN,NaN,NaN,NaN,NaN,NaN,Butte,310.00,400.50,607.50,814.50,972.50,516.25,590.00,736.25,855.00,972.50,825.00,942.50,1177.50,1366.25,1555.00,1031.25,1177.50,1472.50,1707.50,1943.75,1237.50,1413.75,1767.50,2050.00,2332.50
156080,H,3730,137.00,2.00,9.00,nan,nan,2600.00,nan,3.00,nan,nan,2600.00,42.00,74800.00,4.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2292.50,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25


In [16]:
# create filter masks for each affordability and unit type
ELI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["ELI_studio"])
ELI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["ELI_1_br"])
ELI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["ELI_2_br"])
ELI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["ELI_3_br"])
ELI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["ELI_4_br"])
VLI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["VLI_studio"])
VLI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["VLI_1_br"])
VLI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["VLI_2_br"])
VLI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["VLI_3_br"])
VLI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["VLI_4_br"])
LI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["LI_studio"])
LI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["LI_1_br"])
LI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["LI_2_br"])
LI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["LI_3_br"])
LI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["LI_4_br"])
MI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["MI_studio"])
MI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["MI_1_br"])
MI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["MI_2_br"])
MI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["MI_3_br"])
MI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["MI_4_br"])
MoI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["MoI_studio"])
MoI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["MoI_1_br"])
MoI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["MoI_2_br"])
MoI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["MoI_3_br"])
MoI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["MoI_4_br"])
HI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']>=units_df["MoI_studio"])
HI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']>=units_df["MoI_1_br"])
HI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']>=units_df["MoI_2_br"])
HI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']>=units_df["MoI_3_br"])
HI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']>=units_df["MoI_4_br"])

# apply filter masks to create dummy variables
units_df['rent_ELI_studio_count']=np.where(ELI_studio_mask,1,0)
units_df['rent_ELI_1_br_count']=np.where(ELI_1_br_mask,1,0)
units_df['rent_ELI_2_br_count']=np.where(ELI_2_br_mask,1,0)
units_df['rent_ELI_3_br_count']=np.where(ELI_3_br_mask,1,0)
units_df['rent_ELI_4_br_count']=np.where(ELI_4_br_mask,1,0)
units_df['rent_VLI_studio_count']=np.where(VLI_studio_mask,1,0)
units_df['rent_VLI_1_br_count']=np.where(VLI_1_br_mask,1,0)
units_df['rent_VLI_2_br_count']=np.where(VLI_2_br_mask,1,0)
units_df['rent_VLI_3_br_count']=np.where(VLI_3_br_mask,1,0)
units_df['rent_VLI_4_br_count']=np.where(VLI_4_br_mask,1,0)
units_df['rent_LI_studio_count']=np.where(LI_studio_mask,1,0)
units_df['rent_LI_1_br_count']=np.where(LI_1_br_mask,1,0)
units_df['rent_LI_2_br_count']=np.where(LI_2_br_mask,1,0)
units_df['rent_LI_3_br_count']=np.where(LI_3_br_mask,1,0)
units_df['rent_LI_4_br_count']=np.where(LI_4_br_mask,1,0)
units_df['rent_MI_studio_count']=np.where(MI_studio_mask,1,0)
units_df['rent_MI_1_br_count']=np.where(MI_1_br_mask,1,0)
units_df['rent_MI_2_br_count']=np.where(MI_2_br_mask,1,0)
units_df['rent_MI_3_br_count']=np.where(MI_3_br_mask,1,0)
units_df['rent_MI_4_br_count']=np.where(MI_4_br_mask,1,0)
units_df['rent_MoI_studio_count']=np.where(MoI_studio_mask,1,0)
units_df['rent_MoI_1_br_count']=np.where(MoI_1_br_mask,1,0)
units_df['rent_MoI_2_br_count']=np.where(MoI_2_br_mask,1,0)
units_df['rent_MoI_3_br_count']=np.where(MoI_3_br_mask,1,0)
units_df['rent_MoI_4_br_count']=np.where(MoI_4_br_mask,1,0)
units_df['rent_HI_studio_count']=np.where(HI_studio_mask,1,0)
units_df['rent_HI_1_br_count']=np.where(HI_1_br_mask,1,0)
units_df['rent_HI_2_br_count']=np.where(HI_2_br_mask,1,0)
units_df['rent_HI_3_br_count']=np.where(HI_3_br_mask,1,0)
units_df['rent_HI_4_br_count']=np.where(HI_4_br_mask,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count
0,H,10702,102.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,160000.00,nan,nan,54000.00,6.00,25.00,Tulare CA,NaN,NaN,NaN,NaN,NaN,NaN,Tulare,310.00,400.50,607.50,814.50,972.50,516.25,590.00,736.25,855.00,972.50,825.00,942.50,1177.50,1366.25,1555.00,1031.25,1177.50,1472.50,1707.50,1943.75,1237.50,1413.75,1767.50,2050.00,2332.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H,6514,122.00,4.00,2.00,0.00,1000.00,nan,nan,1.00,nan,290000.00,nan,nan,41500.00,7.00,28.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,607.50,814.50,1022.25,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,H,3759,58.00,3.00,2.00,0.00,2000.00,nan,nan,1.00,nan,700000.00,nan,nan,204000.00,3.00,65.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2292.50,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,H,5917,73.00,2.00,2.00,300.00,2100.00,nan,nan,1.00,nan,400000.00,nan,nan,130000.00,1.00,30.00,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,512.50,585.00,731.25,848.75,1022.25,853.75,975.00,1218.75,1413.75,1608.75,1365.00,1560.00,1950.00,2262.50,2575.00,1526.25,1743.75,2180.00,2528.75,2877.50,1831.25,2092.50,2616.25,3035.00,3453.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H,3739,52.00,1.00,6.00,nan,nan,1100.00,nan,3.00,nan,nan,1150.00,18.00,76000.00,3.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2292.50,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156077,H,3753,69.00,2.00,2.00,0.00,1300.00,nan,nan,1.00,nan,350000.00,nan,nan,34800.00,6.00,38.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2292.50,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
156078,H,3734,45.00,1.00,8.00,nan,nan,700.00,nan,3.00,nan,nan,780.00,21.00,44300.00,7.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.5

## Part B1. Calculate total monthly housing cost by adding monthly first mortgage, second/junior mortgage, and condo fees

In [17]:
units_df['own_monthly_cost']=units_df['first_mortgage'] + units_df['second_mortgage'] + units_df['condo_fee'] + (units_df['prop_tax']/12)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost
0,H,10702,102.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,160000.00,nan,nan,54000.00,6.00,25.00,Tulare CA,NaN,NaN,NaN,NaN,NaN,NaN,Tulare,310.00,400.50,607.50,814.50,972.50,516.25,590.00,736.25,855.00,972.50,825.00,942.50,1177.50,1366.25,1555.00,1031.25,1177.50,1472.50,1707.50,1943.75,1237.50,1413.75,1767.50,2050.00,2332.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
1,H,6514,122.00,4.00,2.00,0.00,1000.00,nan,nan,1.00,nan,290000.00,nan,nan,41500.00,7.00,28.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,607.50,814.50,1022.25,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
2,H,3759,58.00,3.00,2.00,0.00,2000.00,nan,nan,1.00,nan,700000.00,nan,nan,204000.00,3.00,65.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2292.50,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
3,H,5917,73.00,2.00,2.00,300.00,2100.00,nan,nan,1.00,nan,400000.00,nan,nan,130000.00,1.00,30.00,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,512.50,585.00,731.25,848.75,1022.25,853.75,975.00,1218.75,1413.75,1608.75,1365.00,1560.00,1950.00,2262.50,2575.00,1526.25,1743.75,2180.00,2528.75,2877.50,1831.25,2092.50,2616.25,3035.00,3453.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
4,H,3739,52.00,1.00,6.00,nan,nan,1100.00,nan,3.00,nan,nan,1150.00,18.00,76000.00,3.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2292.50,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156077,H,3753,69.00,2.00,2.00,0.00,1300.00,nan,nan,1.00,nan,350000.00,nan,nan,34800.00,6.00,38.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2292.50,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
156078,H,3734,45.00,1.00,8.00,nan,nan,700.00,nan,3.00,nan,nan,780.00,21.00,44300.00,7.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50

# Part B2. Add count dummies for each unit based on income and unit type

In [18]:
# create filter masks for each affordability and unit type
ELI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["ELI_studio"])
ELI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["ELI_1_br"])
ELI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["ELI_2_br"])
ELI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["ELI_3_br"])
ELI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["ELI_4_br"])
VLI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["VLI_studio"])
VLI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["VLI_1_br"])
VLI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["VLI_2_br"])
VLI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["VLI_3_br"])
VLI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["VLI_4_br"])
LI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["LI_studio"])
LI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["LI_1_br"])
LI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["LI_2_br"])
LI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["LI_3_br"])
LI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["LI_4_br"])
MI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["MI_studio"])
MI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["MI_1_br"])
MI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["MI_2_br"])
MI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["MI_3_br"])
MI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["MI_4_br"])
MoI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["MoI_studio"])
MoI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["MoI_1_br"])
MoI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["MoI_2_br"])
MoI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["MoI_3_br"])
MoI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["MoI_4_br"])
HI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']>=units_df["MoI_studio"])
HI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']>=units_df["MoI_1_br"])
HI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']>=units_df["MoI_2_br"])
HI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']>=units_df["MoI_3_br"])
HI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']>=units_df["MoI_4_br"])

# apply filter masks to create dummy variables
units_df['own_ELI_studio_count']=np.where(ELI_studio_mask,1,0)
units_df['own_ELI_1_br_count']=np.where(ELI_1_br_mask,1,0)
units_df['own_ELI_2_br_count']=np.where(ELI_2_br_mask,1,0)
units_df['own_ELI_3_br_count']=np.where(ELI_3_br_mask,1,0)
units_df['own_ELI_4_br_count']=np.where(ELI_4_br_mask,1,0)
units_df['own_VLI_studio_count']=np.where(VLI_studio_mask,1,0)
units_df['own_VLI_1_br_count']=np.where(VLI_1_br_mask,1,0)
units_df['own_VLI_2_br_count']=np.where(VLI_2_br_mask,1,0)
units_df['own_VLI_3_br_count']=np.where(VLI_3_br_mask,1,0)
units_df['own_VLI_4_br_count']=np.where(VLI_4_br_mask,1,0)
units_df['own_LI_studio_count']=np.where(LI_studio_mask,1,0)
units_df['own_LI_1_br_count']=np.where(LI_1_br_mask,1,0)
units_df['own_LI_2_br_count']=np.where(LI_2_br_mask,1,0)
units_df['own_LI_3_br_count']=np.where(LI_3_br_mask,1,0)
units_df['own_LI_4_br_count']=np.where(LI_4_br_mask,1,0)
units_df['own_MI_studio_count']=np.where(MI_studio_mask,1,0)
units_df['own_MI_1_br_count']=np.where(MI_1_br_mask,1,0)
units_df['own_MI_2_br_count']=np.where(MI_2_br_mask,1,0)
units_df['own_MI_3_br_count']=np.where(MI_3_br_mask,1,0)
units_df['own_MI_4_br_count']=np.where(MI_4_br_mask,1,0)
units_df['own_MoI_studio_count']=np.where(MoI_studio_mask,1,0)
units_df['own_MoI_1_br_count']=np.where(MoI_1_br_mask,1,0)
units_df['own_MoI_2_br_count']=np.where(MoI_2_br_mask,1,0)
units_df['own_MoI_3_br_count']=np.where(MoI_3_br_mask,1,0)
units_df['own_MoI_4_br_count']=np.where(MoI_4_br_mask,1,0)
units_df['own_HI_studio_count']=np.where(HI_studio_mask,1,0)
units_df['own_HI_1_br_count']=np.where(HI_1_br_mask,1,0)
units_df['own_HI_2_br_count']=np.where(HI_2_br_mask,1,0)
units_df['own_HI_3_br_count']=np.where(HI_3_br_mask,1,0)
units_df['own_HI_4_br_count']=np.where(HI_4_br_mask,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
0,H,10702,102.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,160000.00,nan,nan,54000.00,6.00,25.00,Tulare CA,NaN,NaN,NaN,NaN,NaN,NaN,Tulare,310.00,400.50,607.50,814.50,972.50,516.25,590.00,736.25,855.00,972.50,825.00,942.50,1177.50,1366.25,1555.00,1031.25,1177.50,1472.50,1707.50,1943.75,1237.50,1413.75,1767.50,2050.00,2332.50,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H,6514,122.00,4.00,2.00,0.00,1000.00,nan,nan,1.00,nan,290000.00,nan,nan,41500.00,7.00,28.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,607.50,814.50,1022.25,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,H,3759,58.00,3.00,2.00,0.00,2000.00,nan,nan,1.00,nan,700000.00,nan,nan,204000.00,3.00,65.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2292.50,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,H,5917,73.00,2.00,2.00,300.00,2100.00,nan,nan,1.00,nan,400000.00,nan,nan,130000.00,1.00,30.00,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,512.50,585.00,731.25,848.75,1022.25,853.75,975.00,1218.75,1413.75,1608.75,1365.00,1560.00,1950.00,2262.50,2575.00,1526.25,1743.75,2180.00,2528.75,2877.50,1831.25,2092.50,2616.25,3035.00,3453.75,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H,3739,52.00,1.00,6.00,nan,nan,1100.00,nan,3.00,nan,nan,1150.00,18.00,76000.00,3.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2292.50,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

# Part C1. Multiply each dummy by the the household weight variable

In [19]:
units_df.rent_ELI_studio_count=units_df.rent_ELI_studio_count*units_df.weight
units_df.rent_ELI_1_br_count=units_df.rent_ELI_1_br_count*units_df.weight
units_df.rent_ELI_2_br_count=units_df.rent_ELI_2_br_count*units_df.weight
units_df.rent_ELI_3_br_count=units_df.rent_ELI_3_br_count*units_df.weight
units_df.rent_ELI_4_br_count=units_df.rent_ELI_4_br_count*units_df.weight
units_df.rent_VLI_studio_count=units_df.rent_VLI_studio_count*units_df.weight
units_df.rent_VLI_1_br_count=units_df.rent_VLI_1_br_count*units_df.weight
units_df.rent_VLI_2_br_count=units_df.rent_VLI_2_br_count*units_df.weight
units_df.rent_VLI_3_br_count=units_df.rent_VLI_3_br_count*units_df.weight
units_df.rent_VLI_4_br_count=units_df.rent_VLI_4_br_count*units_df.weight
units_df.rent_LI_studio_count=units_df.rent_LI_studio_count*units_df.weight
units_df.rent_LI_1_br_count=units_df.rent_LI_1_br_count*units_df.weight
units_df.rent_LI_2_br_count=units_df.rent_LI_2_br_count*units_df.weight
units_df.rent_LI_3_br_count=units_df.rent_LI_3_br_count*units_df.weight
units_df.rent_LI_4_br_count=units_df.rent_LI_4_br_count*units_df.weight
units_df.rent_MI_studio_count=units_df.rent_MI_studio_count*units_df.weight
units_df.rent_MI_1_br_count=units_df.rent_MI_1_br_count*units_df.weight
units_df.rent_MI_2_br_count=units_df.rent_MI_2_br_count*units_df.weight
units_df.rent_MI_3_br_count=units_df.rent_MI_3_br_count*units_df.weight
units_df.rent_MI_4_br_count=units_df.rent_MI_4_br_count*units_df.weight
units_df.rent_MoI_studio_count=units_df.rent_MoI_studio_count*units_df.weight
units_df.rent_MoI_1_br_count=units_df.rent_MoI_1_br_count*units_df.weight
units_df.rent_MoI_2_br_count=units_df.rent_MoI_2_br_count*units_df.weight
units_df.rent_MoI_3_br_count=units_df.rent_MoI_3_br_count*units_df.weight
units_df.rent_MoI_4_br_count=units_df.rent_MoI_4_br_count*units_df.weight
units_df.rent_HI_studio_count=units_df.rent_HI_studio_count*units_df.weight
units_df.rent_HI_1_br_count=units_df.rent_HI_1_br_count*units_df.weight
units_df.rent_HI_2_br_count=units_df.rent_HI_2_br_count*units_df.weight
units_df.rent_HI_3_br_count=units_df.rent_HI_3_br_count*units_df.weight
units_df.rent_HI_4_br_count=units_df.rent_HI_4_br_count*units_df.weight
units_df.own_ELI_studio_count=units_df.own_ELI_studio_count*units_df.weight
units_df.own_ELI_1_br_count=units_df.own_ELI_1_br_count*units_df.weight
units_df.own_ELI_2_br_count=units_df.own_ELI_2_br_count*units_df.weight
units_df.own_ELI_3_br_count=units_df.own_ELI_3_br_count*units_df.weight
units_df.own_ELI_4_br_count=units_df.own_ELI_4_br_count*units_df.weight
units_df.own_VLI_studio_count=units_df.own_VLI_studio_count*units_df.weight
units_df.own_VLI_1_br_count=units_df.own_VLI_1_br_count*units_df.weight
units_df.own_VLI_2_br_count=units_df.own_VLI_2_br_count*units_df.weight
units_df.own_VLI_3_br_count=units_df.own_VLI_3_br_count*units_df.weight
units_df.own_VLI_4_br_count=units_df.own_VLI_4_br_count*units_df.weight
units_df.own_LI_studio_count=units_df.own_LI_studio_count*units_df.weight
units_df.own_LI_1_br_count=units_df.own_LI_1_br_count*units_df.weight
units_df.own_LI_2_br_count=units_df.own_LI_2_br_count*units_df.weight
units_df.own_LI_3_br_count=units_df.own_LI_3_br_count*units_df.weight
units_df.own_LI_4_br_count=units_df.own_LI_4_br_count*units_df.weight
units_df.own_MI_studio_count=units_df.own_MI_studio_count*units_df.weight
units_df.own_MI_1_br_count=units_df.own_MI_1_br_count*units_df.weight
units_df.own_MI_2_br_count=units_df.own_MI_2_br_count*units_df.weight
units_df.own_MI_3_br_count=units_df.own_MI_3_br_count*units_df.weight
units_df.own_MI_4_br_count=units_df.own_MI_4_br_count*units_df.weight
units_df.own_MoI_studio_count=units_df.own_MoI_studio_count*units_df.weight
units_df.own_MoI_1_br_count=units_df.own_MoI_1_br_count*units_df.weight
units_df.own_MoI_2_br_count=units_df.own_MoI_2_br_count*units_df.weight
units_df.own_MoI_3_br_count=units_df.own_MoI_3_br_count*units_df.weight
units_df.own_MoI_4_br_count=units_df.own_MoI_4_br_count*units_df.weight
units_df.own_HI_studio_count=units_df.own_HI_studio_count*units_df.weight
units_df.own_HI_1_br_count=units_df.own_HI_1_br_count*units_df.weight
units_df.own_HI_2_br_count=units_df.own_HI_2_br_count*units_df.weight
units_df.own_HI_3_br_count=units_df.own_HI_3_br_count*units_df.weight
units_df.own_HI_4_br_count=units_df.own_HI_4_br_count*units_df.weight
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
0,H,10702,102.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,160000.00,nan,nan,54000.00,6.00,25.00,Tulare CA,NaN,NaN,NaN,NaN,NaN,NaN,Tulare,310.00,400.50,607.50,814.50,972.50,516.25,590.00,736.25,855.00,972.50,825.00,942.50,1177.50,1366.25,1555.00,1031.25,1177.50,1472.50,1707.50,1943.75,1237.50,1413.75,1767.50,2050.00,2332.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,H,6514,122.00,4.00,2.00,0.00,1000.00,nan,nan,1.00,nan,290000.00,nan,nan,41500.00,7.00,28.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,607.50,814.50,1022.25,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,H,3759,58.00,3.00,2.00,0.00,2000.00,nan,nan,1.00,nan,700000.00,nan,nan,204000.00,3.00,65.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2292.50,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,H,5917,73.00,2.00,2.00,300.00,2100.00,nan,nan,1.00,nan,400000.00,nan,nan,130000.00,1.00,30.00,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,512.50,585.00,731.25,848.75,1022.25,853.75,975.00,1218.75,1413.75,1608.75,1365.00,1560.00,1950.00,2262.50,2575.00,1526.25,1743.75,2180.00,2528.75,2877.50,1831.25,2092.50,2616.25,3035.00,3453.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,H,3739,52.00,1.00,6.00,nan,nan,1100.00,nan,3.00,nan,nan,1150.00,18.00,76000.00,3.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258

In [20]:
units_df[units_df.rent_HI_4_br_count>0]

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
710,H,3728,97.00,4.00,2.00,nan,nan,2800.00,nan,3.00,nan,nan,3120.00,31.00,119700.00,1.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2292.50,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,97.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1575,H,3746,96.00,4.00,3.00,nan,nan,3000.00,nan,3.00,nan,nan,3000.00,nan,0.00,1.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2292.50,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,96.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1590,H,3736,53.00,4.00,3.00,nan,nan,2800.00,nan,3.00,nan,nan,2920.00,101.00,25000.00,4.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2292.50,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,53.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2314,H,7309,62.00,4.00,2.00,nan,nan,3800.00,nan,3.00,nan,nan,4150.00,50.00,100000.00,2.00,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,446.25,510.00,637.50,814.50,1022.25,743.75,850.00,1062.50,1232.50,1402.50,1190.00,1360.00,1700.00,1972.50,2245.00,1328.75,1517.50,1897.50,2201.25,2505.00,1593.75,1822.50,2277.50,2642.50,3006.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,62.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2600,H,3725,89.00,4.00,2.00,nan,nan,3100.00,nan,3.00,nan,nan,3763.00,39.00,116000.00,2.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022

# Part D1. Add dummy variables for vacant "for-rent," vacant "for-sale," and recently moved in

In [21]:
# create dummy variables for vacant for rent, vacant for sale, rented in the last year, and sold in the last year
units_df["vacant_for_rent"]=np.where(units_df['vacancy_status']==1,1,0)
units_df["vacant_for_sale"]=np.where(units_df['vacancy_status']==3,1,0)
units_df["moved_last_year"]=np.where(units_df['moved_in']==1,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count,vacant_for_rent,vacant_for_sale,moved_last_year
0,H,10702,102.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,160000.00,nan,nan,54000.00,6.00,25.00,Tulare CA,NaN,NaN,NaN,NaN,NaN,NaN,Tulare,310.00,400.50,607.50,814.50,972.50,516.25,590.00,736.25,855.00,972.50,825.00,942.50,1177.50,1366.25,1555.00,1031.25,1177.50,1472.50,1707.50,1943.75,1237.50,1413.75,1767.50,2050.00,2332.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
1,H,6514,122.00,4.00,2.00,0.00,1000.00,nan,nan,1.00,nan,290000.00,nan,nan,41500.00,7.00,28.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,607.50,814.50,1022.25,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
2,H,3759,58.00,3.00,2.00,0.00,2000.00,nan,nan,1.00,nan,700000.00,nan,nan,204000.00,3.00,65.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2292.50,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
3,H,5917,73.00,2.00,2.00,300.00,2100.00,nan,nan,1.00,nan,400000.00,nan,nan,130000.00,1.00,30.00,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,512.50,585.00,731.25,848.75,1022.25,853.75,975.00,1218.75,1413.75,1608.75,1365.00,1560.00,1950.00,2262.50,2575.00,1526.25,1743.75,2180.00,2528.75,2877.50,1831.25,2092.50,2616.25,3035.00,3453.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
4,H,3739,52.00,1.00,6.00,nan,nan,1100.00,nan,3.00,nan,nan,1150.00,18.00,76000.00,3.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,456.25,521.25,651.25,814.50,1022.25,760.00,868.75,1085.00,1258.75,1432.50,1216.25,1390.00,1736.25,2015.00,2

In [22]:
#filter for only currently listed or recently moved into units
recent_mask = (units_df.vacant_for_rent==1)|(units_df.vacant_for_sale==1)|(units_df.moved_last_year==1)
recent_units_df = units_df[recent_mask]
recent_units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count,vacant_for_rent,vacant_for_sale,moved_last_year
3,H,5917,73.00,2.00,2.00,300.00,2100.00,nan,nan,1.00,nan,400000.00,nan,nan,130000.00,1.00,30.00,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,512.50,585.00,731.25,848.75,1022.25,853.75,975.00,1218.75,1413.75,1608.75,1365.00,1560.00,1950.00,2262.50,2575.00,1526.25,1743.75,2180.00,2528.75,2877.50,1831.25,2092.50,2616.25,3035.00,3453.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
12,H,2500,79.00,1.00,1.00,nan,nan,730.00,nan,3.00,nan,nan,810.00,5.00,191500.00,1.00,nan,Imperial CA,NaN,NaN,NaN,NaN,NaN,NaN,Imperial,310.00,400.50,607.50,814.50,972.50,516.25,590.00,736.25,855.00,972.50,825.00,942.50,1177.50,1366.25,1555.00,1031.25,1177.50,1472.50,1707.50,1943.75,1237.50,1413.75,1767.50,2050.00,2332.50,...,0.00,0.00,79.00,0.00,0.00,0.00,0.00,79.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
20,H,1906,85.00,3.00,2.00,0.00,1200.00,nan,nan,1.00,nan,500000.00,nan,nan,134200.00,1.00,38.00,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno,310.00,400.50,607.50,814.50,972.50,516.25,590.00,736.25,855.00,972.50,825.00,942.50,1177.50,1366.25,1555.00,1031.25,1177.50,1472.50,1707.50,1943.75,1237.50,1413.75,1767.50,2050.00,2332.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
21,H,8505,114.00,3.00,3.00,0.00,nan,nan,nan,2.00,nan,700000.00,nan,nan,50000.00,1.00,52.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,586.25,670.00,837.50,972.50,1106.25,977.50,1116.25,1395.00,1618.75,1842.50,1485.00,1697.50,2122.50,2461.25,2801.25,1873.75,2142.50,2677.50,3106.25,3533.75,2248.75,2570.00,3212.50,3726.25,4240.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
35,H,9501,182.00,1.00,5.00,nan,nan,1700.00,nan,3.00,nan,nan,1988.00,20.00,118000.00,1.00,nan,Solano CA,NaN,NaN,NaN,NaN,NaN,NaN,Solano,435.00,496.25,620.00,814.50,1022.25,723.75,826.25,1032.50,1198.75,1363.75,1137.50,1300.00,1625.00,1885.00,2145.00,1445.0

# Part D2. Aggregating at the PUMA level

In [23]:
# for all units
puma_sums_df = units_df.groupby("PUMA").sum()
puma_counts_df = puma_sums_df[["rent_ELI_studio_count","rent_ELI_1_br_count",
    "rent_ELI_2_br_count",
    "rent_ELI_3_br_count",
    "rent_ELI_4_br_count",
    "rent_VLI_studio_count",
    "rent_VLI_1_br_count",
    "rent_VLI_2_br_count",
    "rent_VLI_3_br_count",
    "rent_VLI_4_br_count",
    "rent_LI_studio_count",
    "rent_LI_1_br_count",
    "rent_LI_2_br_count",
    "rent_LI_3_br_count",
    "rent_LI_4_br_count",
    "rent_MI_studio_count",
    "rent_MI_1_br_count",
    "rent_MI_2_br_count",
    "rent_MI_3_br_count",
    "rent_MI_4_br_count",
    "rent_MoI_studio_count",
    "rent_MoI_1_br_count",
    "rent_MoI_2_br_count",
    "rent_MoI_3_br_count",
    "rent_MoI_4_br_count",
    "rent_HI_studio_count",
    "rent_HI_1_br_count",
    "rent_HI_2_br_count",
    "rent_HI_3_br_count",
    "rent_HI_4_br_count",
    "own_ELI_studio_count",
    "own_ELI_1_br_count",
    "own_ELI_2_br_count",
    "own_ELI_3_br_count",
    "own_ELI_4_br_count",
    "own_VLI_studio_count",
    "own_VLI_1_br_count",
    "own_VLI_2_br_count",
    "own_VLI_3_br_count",
    "own_VLI_4_br_count",
    "own_LI_studio_count",
    "own_LI_1_br_count",
    "own_LI_2_br_count",
    "own_LI_3_br_count",
    "own_LI_4_br_count",
    "own_MI_studio_count",
    "own_MI_1_br_count",
    "own_MI_2_br_count",
    "own_MI_3_br_count",
    "own_MI_4_br_count",
    "own_MoI_studio_count",
    "own_MoI_1_br_count",
    "own_MoI_2_br_count",
    "own_MoI_3_br_count",
    "own_MoI_4_br_count",
    "own_HI_studio_count",
    "own_HI_1_br_count",
    "own_HI_2_br_count",
    "own_HI_3_br_count",
    "own_HI_4_br_count"]].copy()
puma_counts_df

,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
PUMA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
101,366.00,701.00,366.00,438.00,278.00,1057.00,2607.00,2663.00,678.00,696.00,2304.00,6254.00,6745.00,1380.00,740.00,3211.00,8033.00,9217.00,1858.00,1036.00,3840.00,9304.00,10457.00,2057.00,1179.00,507.00,603.00,1499.00,766.00,94.00,0.00,0.00,0.00,29.00,63.00,0.00,0.00,58.00,69.00,63.00,0.00,0.00,291.00,424.00,63.00,0.00,0.00,591.00,484.00,63.00,0.00,0.00,710.00,681.00,220.00,0.00,0.00,385.00,365.00,386.00
102,1611.00,5184.00,2447.00,1864.00,461.00,5164.00,8920.00,6307.00,3757.00,671.00,8664.00,15489.00,9977.00,5460.00,881.00,9641.00,18406.00,12684.00,5917.00,1212.00,10076.00,21999.00,13533.00,6093.00,1251.00,1416.00,2770.00,1081.00,89.00,45.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,91.00,52.00,0.00,0.00,0.00,133.00,52.00,0.00,391.00,148.00,298.00,126.00,0.00,391.00,148.00,374.00,182.00,0.00,222.00,95.00,0.00,0.00
103,374.00,1267.00,184.00,0.00,0.00,483.00,1869.00,1551.00,257.00,77.00,1604.00,5385.00,3455.00,1122.00,186.00,1643.00,6348.00,4137.00,1584.00,331.00,1643.00,7218.00,5017.00,2102.00,331.00,93.00,592.00,571.00,253.00,50.00,0.00,0.00,0.00,57.00,0.00,0.00,0.00,60.00,204.00,141.00,0.00,0.00,224.00,546.00,315.00,0.00,0.00,302.00,1214.00,485.00,0.00,0.00,614.00,1595.00,485.00,0.00,142.00,275.00,493.00,263.00
104,137.00,874.00,1723.00,983.00,66.00,876.00,2561.00,5969.00,2620.00,679.00,1856.00,4730.00,10395.00,5631.00,935.00,1939.00,4730.00,10520.00,5857.00,935.00,2006.00,4730.00,10647.00,5928.00,935.00,81.00,0.00,0.00,67.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,109.00,0.00,0.00,0.00,110.00,315.00,0.00,0.00,0.00,110.00,350.00,0.00,0.00,0.00,110.00,350.00,0.00,0.00,0.00,305.00,0.00,0.00
105,217.00,1232.00,726.00,636.00,677.00,357.00,3046.00,2866.00,2091.00,746.00,1053.00,7484.00,8737.00,3288.00,872.00,1912.00,9207.00,11879.00,4033.00,992.00,2055.00,9772.00,12803.00,4161.00,992.00,57.00,591.00,350.00,0.00,0.00,0.00,0.00,0.00,213.00,0.00,0.00,0.00,212.00,687.00,93.00,0.00,0.00,373.00,943.00,390.00,0.00,0.00,705.00,1730.00,493.00,0.00,0.00,890.00,2230.00,733.00,0.00,0.00,540.00,242.00,254.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11103,0.00,840.00,484.00,411.00,361.00,98.00,2366.00,2810.00,990.00,669.00,340.00,4904.00,10124.00,3724.00,2437.00,375.00,5255.00,11311.00,4310.00,2836.00,693.00,5427.00,11311.00,4556.00,2836.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,38.00,35.00,0.00,0.00,94.00,408.00,228.00,0.00,0.00,94.00,457.00,350.00,0.00,0.00,206.00,568.00,350.00,0.00,0.00,0.00,40.00,0.00
11104,328.00,531.00,607.00,330.00,176.00,699.00,761.00,1311.00,678.00,176.00,1449.00,2897.00,7332.00,2389

In [24]:
#export it
puma_counts_df.to_csv("weighted_puma_counts.csv")

In [25]:
# for recent units
recent_puma_sums_df = recent_units_df.groupby("PUMA").sum()
recent_puma_counts_df = recent_puma_sums_df[["rent_ELI_studio_count","rent_ELI_1_br_count",
    "rent_ELI_2_br_count",
    "rent_ELI_3_br_count",
    "rent_ELI_4_br_count",
    "rent_VLI_studio_count",
    "rent_VLI_1_br_count",
    "rent_VLI_2_br_count",
    "rent_VLI_3_br_count",
    "rent_VLI_4_br_count",
    "rent_LI_studio_count",
    "rent_LI_1_br_count",
    "rent_LI_2_br_count",
    "rent_LI_3_br_count",
    "rent_LI_4_br_count",
    "rent_MI_studio_count",
    "rent_MI_1_br_count",
    "rent_MI_2_br_count",
    "rent_MI_3_br_count",
    "rent_MI_4_br_count",
    "rent_MoI_studio_count",
    "rent_MoI_1_br_count",
    "rent_MoI_2_br_count",
    "rent_MoI_3_br_count",
    "rent_MoI_4_br_count",
    "rent_HI_studio_count",
    "rent_HI_1_br_count",
    "rent_HI_2_br_count",
    "rent_HI_3_br_count",
    "rent_HI_4_br_count",
    "own_ELI_studio_count",
    "own_ELI_1_br_count",
    "own_ELI_2_br_count",
    "own_ELI_3_br_count",
    "own_ELI_4_br_count",
    "own_VLI_studio_count",
    "own_VLI_1_br_count",
    "own_VLI_2_br_count",
    "own_VLI_3_br_count",
    "own_VLI_4_br_count",
    "own_LI_studio_count",
    "own_LI_1_br_count",
    "own_LI_2_br_count",
    "own_LI_3_br_count",
    "own_LI_4_br_count",
    "own_MI_studio_count",
    "own_MI_1_br_count",
    "own_MI_2_br_count",
    "own_MI_3_br_count",
    "own_MI_4_br_count",
    "own_MoI_studio_count",
    "own_MoI_1_br_count",
    "own_MoI_2_br_count",
    "own_MoI_3_br_count",
    "own_MoI_4_br_count",
    "own_HI_studio_count",
    "own_HI_1_br_count",
    "own_HI_2_br_count",
    "own_HI_3_br_count",
    "own_HI_4_br_count"]].copy()
recent_puma_counts_df

,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
PUMA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
101,0.00,70.00,40.00,0.00,0.00,170.00,70.00,517.00,135.00,195.00,551.00,346.00,1487.00,135.00,239.00,922.00,941.00,2321.00,135.00,302.00,1426.00,1611.00,2972.00,198.00,302.00,507.00,279.00,890.00,315.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,71.00
102,120.00,1074.00,336.00,0.00,138.00,478.00,1203.00,823.00,221.00,138.00,1183.00,1402.00,1288.00,361.00,138.00,1441.00,2253.00,2426.00,361.00,138.00,1669.00,3862.00,2908.00,361.00,138.00,865.00,1691.00,436.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
103,0.00,406.00,0.00,0.00,0.00,0.00,406.00,0.00,0.00,77.00,290.00,1055.00,233.00,329.00,186.00,329.00,1482.00,294.00,519.00,186.00,329.00,1654.00,678.00,741.00,186.00,93.00,293.00,372.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,42.00,76.00
104,0.00,0.00,330.00,0.00,0.00,0.00,148.00,538.00,387.00,0.00,598.00,1409.00,1267.00,798.00,0.00,598.00,1409.00,1267.00,1024.00,0.00,665.00,1409.00,1341.00,1024.00,0.00,81.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
105,0.00,0.00,0.00,56.00,81.00,62.00,0.00,457.00,1017.00,81.00,135.00,312.00,457.00,1017.00,81.00,211.00,1012.00,1453.00,1114.00,81.00,285.00,1417.00,2262.00,1114.00,81.00,57.00,474.00,203.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,101.00,0.00,0.00,0.00,0.00,148.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11103,0.00,186.00,0.00,159.00,180.00,0.00,309.00,662.00,159.00,288.00,154.00,1147.00,2878.00,604.00,1231.00,154.00,1314.00,3628.00,709.00,1312.00,154.00,1314.00,3628.00,709.00,1312.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,70.00,0.00,0.00,0.00,0.00,0.00,0.00
11104,0.00,0.00,155.00,0.00,148.00,309.00,0.00,259.00,0.00,148.00,777.00,629.00,2063.00,279.00,148.00,847.00,815.00,2508.00,279.00,227.00,1028.00,1042.00,2816.00,279.00,227.00,66.00,71.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
11105,55.00,0.00,0.00,67.00,0.00,1

In [26]:
#export it
recent_puma_counts_df.to_csv("weighted_recent_puma_counts.csv")